In [2]:
import re
from pathlib import Path

import pandas as pd
import numpy as np
import lifelines
from lifelines.fitters.coxph_fitter import CoxPHFitter

import relife
from relife.lifetime_model import SemiParametricProportionalHazard
from relife.data import csv

Lifelines, even with Breslow baseline estimator, allows estimation beyond the highest time sample seen during training
Lifelines allows 2 other baseline estimator: "spline", or "piecewise"

Other alternatives to Breslow:
https://pmc.ncbi.nlm.nih.gov/articles/PMC6201745/
https://onlinelibrary.wiley.com/doi/abs/10.1002/sim.2574?casa_token=QGxyHGmUnTIAAAAA%3AoLALli8JiXl4dJxjx3r7UQN2YiYm3fdh4q-_zr_gkKCcOXy_vzkNEJbRCpqU3PdJMTrOQE0gebAOEUq-
https://statics.teams.cdn.office.net/evergreen-assets/safelinks/2/atp-safelinks.html

About non-parametric simulation techniques:
https://www.nzdr.ru/data/media/biblio/kolxoz/M/MV/Massart%20P.%20Concentration%20inequalities%20and%20model%20selection%20(LNM1896,%20Springer,%202007)(ISBN%203540484973)(345s)_MV_.pdf

# Take data

## Données chaines d'isolateur

In [ ]:
# Données chaines d'isolateur
relife_csv_datapath = Path(r"D:\Projets\RTE\ReLife\relife\relife\data\csv")
time, event, entry, *args = np.loadtxt(relife_csv_datapath / "insulator_string.csv", delimiter=",", skiprows=1, unpack=True)
covar = np.column_stack(args)

In [ ]:
# Into df
data = pd.DataFrame({"time": time, "event": event, "entry": entry})
covar = pd.DataFrame(covar)
covar.columns = [f"covar_{i}" for i in range(covar.shape[1])]
data = pd.concat([data, covar], axis=1)

## Données Lifelines

In [ ]:
# Import lifelines dataset
data = lifelines.datasets.load_canadian_senators()
print(data.columns)

In [ ]:
# Clean data
data = (
        data
        .rename(
            columns={
                "diff_days": "time",
                "observed": "event",
                "Political Affiliation at Appointment": "affiliation",
                "Province / Territory": "province",
            })
        .drop(
            columns=["Appointed on the advice of", "Term (yyyy.mm.dd)"]
        )
        .set_index("Name")
)
data = data[data["reason"] != 'Appointment declined']
data["province"] = data["province"].str.replace(" ", "_")
data["province"] = data["province"].str.replace("(", "")
data["province"] = data["province"].str.replace(")", "")
print(data.columns)

In [ ]:
# One hot encoding / dropping factor levels with low occurrence
data = data.rename(columns={"province": "covar"})
data = pd.get_dummies(data.set_index("covar", drop=False), columns=["covar"], dtype=int)

province_count = data.filter(regex="covar").sum()
province_with_low_count = province_count[province_count < 10].index
data = data.drop(columns=province_with_low_count)

# Lifelines model fit and sf estimation

In [ ]:
# Lifelines model fit
ll_model = CoxPHFitter()
ll_model.fit(
    df=data,
    duration_col="time",
    event_col="event",
    formula="~ " + " + ".join([c for c in data.columns if re.match("covar", c)])
)
print(ll_model.params_)

In [ ]:
# Lifelines sf
max_offset = 0

X = data.filter(regex="covar").iloc[:2]

sf_lifelines = ll_model.predict_survival_function(
    X=X,
    times=range(data["time"].min().astype(int), data["time"].max().astype(int) + max_offset)
)
sf_lifelines.columns = X.index
sf_lifelines.plot(xlabel="time", ylabel="sf")

# Relife model fit and sf estimation

In [ ]:
# Relife model fit
re_model = Cox()
re_model.fit(
    time=data["time"],
    covar=data.filter(regex="covar").values,
    event=data["event"],
)
print(re_model.params)

In [ ]:
# Relife sf
X = data.filter(regex="covar").iloc[:2]

sf_relife = re_model.sf(
    covar=X,
)
sf_relife = pd.DataFrame(sf_relife.T, columns=X.index)
sf_relife.plot(xlabel="time", ylabel="sf")